# Distil-BERT, RawText + Aspect to SentimentPolarity

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.0 MB/s eta 0:00:00


In [ ]:
import os
import tensorflow as tf
import pandas as pd
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
target_data_path = "train 데이터 위치를 입력합니다."
model_save_path = "모델이 저장될 위치를 입력합니다."

In [ ]:
base_df = pd.read_csv(target_data_path).dropna()
base_df.head(2)

,Index,RawText,Source,Domain,MainCategory,ProductName,ReviewScore,Syllable,Word,RDate,GeneralPolarity,Aspect,SentimentText,SentimentWord,SentimentPolarity
0,128484,이번에구매한데님은사이즈가잘맞네요 색상구성도괜찮고맘에든답니다 잘입겠습니다,쇼핑몰,패션,남성의류,OO 남성 매** 데님 3종,100,39,3,20180315,1.0,사이즈,사이즈가잘맞네요,1,1
1,128494,바지는 너무 편하고 좋은데 좀크게나온듯 그리고 허리고리 하나가 안달려서 밑단수선하면...,쇼핑몰,패션,남성의류,OO 남성 매** 데님 3종,60,118,24,20180317,0.0,사이즈,좀크게나온듯,1,-1


In [ ]:
target_df = base_df.loc[:, ["RawText", "Aspect", "SentimentPolarity"]]
target_df.head(3)

,RawText,Aspect,SentimentPolarity
0,이번에구매한데님은사이즈가잘맞네요 색상구성도괜찮고맘에든답니다 잘입겠습니다,사이즈,1
1,바지는 너무 편하고 좋은데 좀크게나온듯 그리고 허리고리 하나가 안달려서 밑단수선하면...,사이즈,-1
2,신랑 바지가 없어서 사줬는데 엄청 입기 편하다고 좋아하네요 똑같은 라인으로 여성용 ...,가격,1


In [ ]:
num_labels = len(target_df["SentimentPolarity"].unique())
num_labels

3

In [ ]:
MODEL_PATH = "distilbert-base-multilingual-cased"
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=num_labels, from_pt=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'cla

In [ ]:
test_row = target_df.loc[0, :]
print(test_row)
test_encoding = tokenizer.encode_plus(
    text=test_row["RawText"],
    text_pair=test_row["Aspect"],
    padding=True,
    truncation=True,
    max_length=42,
    add_special_tokens=True,
    return_token_type_ids=True,
    return_tensors="tf"
)
test_encoding

RawText              이번에구매한데님은사이즈가잘맞네요 색상구성도괜찮고맘에든답니다 잘입겠습니다
Aspect                                                   사이즈
SentimentPolarity                                          1
Name: 0, dtype: object


{'input_ids': <tf.Tensor: shape=(1, 41), dtype=int32, numpy=
array([[   101,   9638,  35465,  10530,  17196, 100372,  11102,  28911,
        108578,  10892,  12945,  10739,  24891,  11287, 119197, 118913,
         77884,  48549,   9416,  14871,  17196,  17138,  12092, 118650,
        119250,  11664, 118911,  10530,  90537, 118775,  48345,   9654,
         58303, 118632, 119081,  48345,    102,   9405,  10739,  24891,
           102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 41), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 41), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
      dtype=int32)>}

In [ ]:
X_train_encoding = tokenizer(
    text=target_df["RawText"].to_list(),
    text_pair=target_df["Aspect"].to_list(),
    padding=True,
    truncation=True,
    max_length=42,
    # return_token_type_ids: True
)

dict(X_train_encoding).keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
dict(X_train_encoding)["input_ids"][:3]

In [ ]:
# X_train_encoding = {
#     "input_ids": [],
#     "token_type_ids": [],
#     "attention_mask": []
# }
# for idx, row in target_df.iterrows():
#     each_encoding = tokenizer.encode_plus(
#         text=row["RawText"],
#         text_pair=row["Aspect"],
#         padding="max_length",
#         truncation=True,
#         max_length=42,
#         add_special_tokens=True,
#         return_token_type_ids=True,
#         return_tensors="tf"
#     )
#     for key in "input_ids", "token_type_ids", "attention_mask":
#         X_train_encoding[key].append(each_encoding[key])

# X_train_encoding["input_ids"][:3]

In [ ]:
SHUFFLE_PARAM = 1000

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train_encoding),
    target_df["SentimentPolarity"].to_numpy() + 1
)).shuffle(SHUFFLE_PARAM)

In [ ]:
BATCH_PARAM = 32

validation_length = int(len(target_df) // 10)
train_except_val = train_dataset.skip(validation_length).batch(BATCH_PARAM)
validation_data = train_dataset.take(validation_length).batch(BATCH_PARAM)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, metrics=["accuracy"])
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 134734080 
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 135,326,979
Trainable params: 135,326,979
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(
    train_except_val,
    epochs=1,
    batch_size=BATCH_PARAM,
    validation_data=validation_data)

1470/1470 [==============================] - 16801s 11s/step - loss: 0.4753 - accuracy: 0.8171 - val_loss: 0.4158 - val_accuracy: 0.8467


In [ ]:
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('/content/drive/MyDrive/zb-2nd/distilbert-aspect-sent/tokenizer_config.json',
 '/content/drive/MyDrive/zb-2nd/distilbert-aspect-sent/special_tokens_map.json',
 '/content/drive/MyDrive/zb-2nd/distilbert-aspect-sent/vocab.txt',
 '/content/drive/MyDrive/zb-2nd/distilbert-aspect-sent/added_tokens.json',
 '/content/drive/MyDrive/zb-2nd/distilbert-aspect-sent/tokenizer.json')